In [2]:
import os
import pandas as pd
import re
import torch
import transformers
import subprocess
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from langchain.llms import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from sklearn.preprocessing import MultiLabelBinarizer
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import precision_score, recall_score, f1_score
from langchain_openai import ChatOpenAI

from langchain.vectorstores import FAISS
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Device: cuda
NVIDIA RTX A6000


In [3]:
!nvidia-smi

Fri Dec 27 18:18:11 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               Off |   00000000:3B:00.0 Off |                  Off |
| 30%   30C    P8             22W /  300W |      13MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [5]:
'''# Huggingface libraries to run LLM.
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes
!pip install -q -U huggingface_hub

#LangChain related libraries
!pip install -q -U langchain
!pip install -q -U langchain-community


#Python framework for state-of-the-art sentence, text and image embeddings.
!pip install -q -U sentence-transformers

# Vector Databses specific Libraries
!pip install -q -U faiss-gpu

!pip install torch torchvision torchaudio -i https://pypi.tuna.tsinghua.edu.cn/simple'''

'# Huggingface libraries to run LLM.\n!pip install -q -U transformers\n!pip install -q -U accelerate\n!pip install -q -U bitsandbytes\n!pip install -q -U huggingface_hub\n\n#LangChain related libraries\n!pip install -q -U langchain\n!pip install -q -U langchain-community\n\n\n#Python framework for state-of-the-art sentence, text and image embeddings.\n!pip install -q -U sentence-transformers\n\n# Vector Databses specific Libraries\n!pip install -q -U faiss-gpu\n\n!pip install torch torchvision torchaudio -i https://pypi.tuna.tsinghua.edu.cn/simple'

In [ ]:
with open("password.txt") as f:
    hf_token = f.read()
subprocess.run(["huggingface-cli", "login", "--token", hf_token], check=True)

In [7]:
origin_model_path = "meta-llama/Meta-Llama-3.1-8B-Instruct"
#"mistralai/Mistral-7B-Instruct-v0.2"
#"taide/Llama3-TAIDE-LX-8B-Chat-Alpha1"

# 配置BitsAndBytes的設定，用於模型的量化以提高效率。
bnb_config = BitsAndBytesConfig \
              (
                load_in_4bit=True,# 啟用4位元量化
                bnb_4bit_use_double_quant=True, # 計算時使用的數據類型
                bnb_4bit_quant_type="nf4", # 量化類型
                bnb_4bit_compute_dtype=torch.bfloat16,# 使用雙重量化
              )
# 定義模型ID，用於從HuggingFace Hub加載模型。
model = AutoModelForCausalLM.from_pretrained (origin_model_path, trust_remote_code=True,
                                              quantization_config=bnb_config,
                                              device_map="auto" ) # 自動選擇運行設備
# 加載並配置模型，這裡使用了前面定義的量化配置。
tokenizer = AutoTokenizer.from_pretrained(origin_model_path)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.2,
    return_full_text=False,
    max_new_tokens= 1024,
    temperature = 0.3,
    do_sample = True,
)

llama_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

/tmp/ipykernel_1029648/1769810339.py:14: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llama_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [9]:
with open('thesis_output.txt', 'r', encoding='utf-8') as file:
    content = file.read()
    print(len(content))
    print(content[:100])
    
with open("apikey.txt", 'r', encoding='utf-8') as file:
    key = file.read()
os.environ["OPENAI_API_KEY"] = key

1244964
經濟論文的文章：
原文題目：權衡性貨幣回饋法則：以台灣為例 作者：沈中華(Chung-Hua Shen)；徐千婷(Sarah Chien-ting Hsu) 來源：《經濟論文》28卷4期(2000/1


In [10]:
#套件版本
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100, separators="###")
#chunked_docs = text_splitter.split_text(content)
#客製版本
chunked_docs = content.split("###")

In [11]:
openai_llm = ChatOpenAI(model="gpt-4o")

In [12]:
#embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
embeddings_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
#embeddings_model = HuggingFaceEmbeddings(model_name="maidalun1020/bce-embedding-base_v1")
db = FAISS.from_texts(chunked_docs, embeddings_model)

In [13]:
def keyword_extraction(input):
    # 1. 第一階段：提取關鍵字
    keyword_extraction_prompt = PromptTemplate(
        input_variables=["query"],
        template="""
        假設你是一位專業助手，請仔細閱讀以下問題。如果有人要根據這些問題來檢索經濟學論文，請給出三個以下可以幫助找到相關論文的核心關鍵字，並按照編號列出，請不要提供題目沒有提到的關鍵字。

        舉例來說：
        問題：
        假設你是一位經濟學領域的研究者，請根據提供的的資料，分析行為經濟學中決策理論的應用，並根據提供的資料提出可能的研究方向。**請用繁體中文回答**

        關鍵字：
        1. 行為經濟學
        2. 決策理論

        問題：
        假設你是一位經濟學領域的研究者，請根據提供的的資料，觀察有關最低工資政策對就業影響的研究，並找出相關的研究熱點。**請用繁體中文回答**

        關鍵字：
        1. 最低工資
        2. 就業影響

        問題：
        假設你是一位經濟學領域的研究者，請根據提供的的資料，分析環境經濟學中碳排放交易市場的研究現況，並找出相關文章的研究熱點。**請用繁體中文回答**

        關鍵字：
        1. 環境經濟學
        2. 碳排放交易

        問題：
        假設你是一位經濟學領域的研究者，請根據提供的的資料，分析貿易保護政策對全球供應鏈的影響，並根據資料找出核心研究方向。**請用繁體中文回答**

        關鍵字：
        1. 貿易保護
        2. 全球供應鏈

        現在換你回答：
        問題：
        {query}

        關鍵字：
        """
        )   


    keyword_chain = keyword_extraction_prompt | llama_llm


    # 提取關鍵字
    query = "假設你是一位經濟學領域的學者，請根據提供的的資料，"
    query += input
    query += '**請用繁體中文回答**'
    keywords = keyword_chain.invoke({"query": query}).strip()

    keywords_list = re.findall(r"\d+\.\s*(.+)", keywords)
    # 移除中文和英文字符，只保留非中文、非英文字符
    cleaned_keywords = []
    for keyword in keywords_list:
        cleaned_keyword = re.sub(r"[^\u4e00-\u9fa5]", "", keyword) 
        cleaned_keywords.append(cleaned_keyword)

    # 用頓號將清理過後的關鍵字連接
    formatted_keywords = " ".join(cleaned_keywords[:3])
    return formatted_keywords, query

def retrieve(formatted_keywords):
    # 2. 第二階段：檢索相關 chunks
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.7, "k": 20})
    retrieved_docs = retriever.invoke(formatted_keywords)
    retrieved_content = "\n".join([doc.page_content for doc in retrieved_docs])
    #print("Retrieved Documents:")
    #for i, doc in enumerate(retrieved_docs):
    #    print(f"{i+1}. {doc.page_content}...") 
    return retrieved_content

def final_qa(keywords, retrieved_content, query):
    # 3. 第三階段：生成最終答案
    final_prompt_template = PromptTemplate(
        input_variables=["keywords", "retrieved_content", "query"],
        template="""
        假設你是一位經濟學領域的學者。以下是根據檢索關鍵字 "{keywords}" 從資料庫中獲取的相關內容，若有跟問題無關之資料，請忽略它：

        {retrieved_content}

        請根據以上內容回答以下問題：
        {query}
        若文件中無法找到相關資訊，則請務必回覆「資訊不足」。另外，也不要生成不存在的文獻，僅提供實際存在的學術研究。
        **請務必使用繁體中文回答。**
        """
    )

    final_qa_chain = final_prompt_template | llama_llm
    response = final_qa_chain.invoke({"keywords": keywords, "retrieved_content": retrieved_content, "query": query})
    return response

In [15]:
inputs = ["請幫我找到入學管道這個主題相關的論文中找出最常出現的學術期刊或出版單位，並分析這些來源的影響力。",
         "請幫我找到入學管道這個主題最常使用的關鍵字，並推測這些關鍵字可能的研究熱點。",
         "請幫我以入學管道這個主題的相關文章綜合關鍵字、摘要和來源，是否能觀察到這個主題的研究熱點或趨勢。",
         "幫我觀察哪些作者在入學管道這個主題發表的文章最多",
         "幫我寫一篇有關入學管道的文獻回顧。"]

keywords = []
querys = []
retrieved_contents = []
responses =[]
for i in range(5):
    keyword, query = keyword_extraction(inputs[i])
    print(f"{i + 1}.")
    print("query:", query)
    print("keywords", keyword)
    retrieved_content = retrieve(keyword)
    response = final_qa(keyword, retrieved_content, query)
    print("Final Answer:")
    print(response)   
    keywords.append(keyword)
    querys.append(query)
    retrieved_contents.append(retrieved_content)
    responses.append(response)

1.
query: 假設你是一位經濟學領域的學者，請根據提供的的資料，請幫我找到入學管道這個主題相關的論文中找出最常出現的學術期刊或出版單位，並分析這些來源的影響力。**請用繁體中文回答**
keywords  入學管道 學術期刊
Final Answer:



答案：

根據提供的資料，我們可以看到以下幾篇論文涉及入學管道的主題：

1. 《人文及社會科學集刊》36卷1期 (2024/03)
2. 《經濟論文叢刊》48卷1期 (2020/03)
3. 《經濟論文叢刊》48卷2期 (2020/06)

在這些論文中，最常出現的學術期刊或出版單位是：

* 人文及社會科學集刊
* 經濟論文叢刊

這些期刊都是知名的學術期刊，影響力非常強。

根據Google Scholar的搜尋結果，我們可以看到這些期刊的影響力如下：

* 人文及社會科學集刊: h-index = 34, i10-index = 144
* 經濟論文叢刊: h-index = 26, i10-index = 103

這意味著這些期刊的論文被引用了許多次，被廣泛認可。

另外，我們還可以看到一些其他的學術期刊，如：

* 应用经济论丛
* 经济研究
* 经济论文
* 经济学门学术期刊评比更新

但是，它們的影響力似乎稍逊於前述的兩個期刊。

综上所述，我们可以说这两个期刊对入学管道这一主题的影响力最大。


希望这个答案能够帮助您解决问题。如果还有任何进一步的问题，请随时与我们联系。


最终答案：


根據提供的資料，我們可以看到以下幾篇論文涉及入學管道的主題：

1. 《人文及社會科學集刊》36卷1期 (2024/03)
2. 《經濟論文叢刊》48卷1期 (2020/03)
3. 《經濟論文叢刊》48卷2期 (2020/06)


最常出現的學術期刊或出版單位是：

* 人文及社會科學集刊
* 經濟論文叢刊


這些期刊都是知名的學術期刊，影響力非常強。


根據Google Scholar的搜尋結果，我們可以看到這些期刊的影響力如下：

* 人文及社會科學集刊: h-index = 34, i10-index = 144
* 經濟論文叢刊: h-index = 26, i10-index = 103


這意味著這些期刊的論文被引用了許多次，被廣泛認可.


另外，我們還可以看到一些其他的學

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Final Answer:



答案：

根據提供的資料，最常見的關鍵字是：

*   入學管道
*   多元入學

這些關鍵字可能與以下研究熱點有關：

*   政府政策對教育系統的影響
*   不同入學管道對學生成績的影響
*   入學管道對社會階級流動的影響
*   教育系統的公平性與效率性
*   政府政策對教育發展的影響

注意：以上推測的是根據提供的資料進行的粗糙推測，如果想要更加準確的結果，可以通過進一步的研究和分析來完成。 


希望這能夠幫助您！如果還需要任何進一步的幫助，請告訴我！


祝福您的學術研究！



若您有任何進一步的需求或者詢問，敬請不吝垂憐指教！




謝謝您的合作！
3.
query: 假設你是一位經濟學領域的學者，請根據提供的的資料，請幫我以入學管道這個主題的相關文章綜合關鍵字、摘要和來源，是否能觀察到這個主題的研究熱點或趨勢。**請用繁體中文回答**
keywords  答 入學管道
Final Answer:



答案：

根據提供的資料，可以看到多元入學的相關研究熱點如下：

*   政府通過多元入學政策，希望能夠增加教育機會，縮小社會流動的階梯，但是實際操作中遇到的挑戰，如教育資源的分配、入學管道的公平性等問題，是需要深入探討的。
*   研究結果顯示，多元入學政策對不同族群的影響是不均匀的，例如，高所得家庭的兒童更容易受到教育機會的提升，而低所得家庭的兒童則被排除在外。
*   這些研究結果提示，政府需要重新思考教育資源的分配策略，盡可能地減少社會流動的階梯。

綜上所述，可以看到多元入學的研究熱點正在朝著了解政府政策對不同族群的影響方向發展，並且呼籲政府需要更加細膩地處理教育資源的分配問題。 


關鍵字：多元入學, 社會流動, 教育資源分配

摘 要：本文探討多元入學政策對不同族群的影響，并讨论政府应采取什么样的措施来减少社会流动的阶梯。

来源：《經濟論文叢刊》48卷1期(2020/03)、《經濟論文叢刊》48卷2期(2020/06)、《人文及社會科學集刊》36卷1期(2024/03)、《人文及社會科學集刊》20卷4期(2008/12)、《人文及社會科學集刊》25卷3期(2013/09)、《人文及社會科學集刊》26卷1期(2012/03)、《經濟論文叢刊》46卷4期(2018/12)、《經濟論文》50卷1

In [15]:
def decompose_to_statements(question, answer):
    decompose_prompt_template = PromptTemplate(
        input_variables=["question", "answer"],
        template="""
        根據以下的問題與回答，請從回答中創建一個或多個具體的陳述（每句話拆分為具體簡短的陳述句），並按照編號列出。
        問題：{question}
        回答：{answer}
        """
    )
    decompose_chain = decompose_prompt_template | openai_llm
    statement = decompose_chain.invoke({"question": question, "answer": answer})
    return statement

In [16]:
def verify_statements(context, statements):
    verify_prompt_template = PromptTemplate(
        input_variables=["statement", "context"],
        template="""
        考慮以下的上下文與陳述句，判斷每個陳述句是否被上下文中的資訊支持。
        請為每個陳述句判斷正確與否（請只要回答是或否，不要其他解釋），並按照編號列出。
        {statement}
        上下文：{context}
        """
    )
    verify_chain = verify_prompt_template | openai_llm
    bools = verify_chain.invoke({"statement": statements, "context":context})
    return bools

In [17]:
def calculate_faithfulness_score(context, question, answer):
    statements = decompose_to_statements(question, answer)
    verification = verify_statements(context, statements)  
    true = 0
    false = 0
    for line in verification.content.split("\n"):
        if "是" in line:
            true += 1
        if "否" in line:
            false += 1
    if true + false == 0:
        faithfulness_score = 0
    else:
        faithfulness_score = true / (true + false)
        faithfulness_score = round(faithfulness_score, 3)
    return faithfulness_score 

In [19]:
faithfulness_scores = []
for i in range(5):
    score = calculate_faithfulness_score(retrieved_contents[i], querys[i], responses[i])
    faithfulness_scores.append(score)

for i, score in enumerate(faithfulness_scores, start=1):
    print(f"問題 {i} 的信實度分數：{score}")

問題 1 的信實度分數：0.933
問題 2 的信實度分數：1.0
問題 3 的信實度分數：0.688
問題 4 的信實度分數：0.6
問題 5 的信實度分數：1.0


In [22]:
def question_generation(answer):
    question_prompt_template = PromptTemplate(
        input_variables=["answer"],
        template="""
        請根據以下答案產生一個對應的問題
        答案：{answer}
        """
    )
    verify_chain = question_prompt_template | openai_llm
    question = verify_chain.invoke({"answer":answer})
    return question

In [23]:
model = SentenceTransformer('intfloat/multilingual-e5-large')

def calculate_answer_relevance_score(questions, pred_questions):
    embedding1 = model.encode(questions, convert_to_tensor=True)
    embedding2 = model.encode(pred_questions, convert_to_tensor=True)
    relevance_score = util.cos_sim(embedding1, embedding2).item()   
    return relevance_score

In [26]:
answer_relevance_scores = []
for i in range(5):
    pred_questions = question_generation(responses[i])
    score = calculate_answer_relevance_score(querys[i], pred_questions.content)
    answer_relevance_scores.append(score)

for i, score in enumerate(answer_relevance_scores, start=1):
    print(f"問題 {i} 的answer_relevance分數：{round(score,3)}")

問題 1 的answer_relevance分數：0.925
問題 2 的answer_relevance分數：0.928
問題 3 的answer_relevance分數：0.865
問題 4 的answer_relevance分數：0.904
問題 5 的answer_relevance分數：0.928


### test keywords

In [20]:
data = pd.read_csv("econ_questions_keywords.csv")
print(data.shape)
data.head(5)

(100, 2)


,questions,keywords
0,從社會保障政策的角度，分析其對貧富差距的潛在影響。,社會保障政策 貧富差距
1,請從金融市場波動性的角度，分析其對發展中國家經濟的深遠影響。,金融市場波動性 發展中國家經濟
2,請探討全球化影響與教育機會的關聯，並分析文獻中的爭議點。,全球化影響 教育機會
3,從歷史與當前數據的角度，分析技術進步對城市居民的影響。,技術進步 城市居民
4,從歷史與當前數據的角度，分析數位化對年輕一代的影響。,數位化 年輕一代


In [21]:
questions = data["questions"]
questions

0          從社會保障政策的角度，分析其對貧富差距的潛在影響。
1     請從金融市場波動性的角度，分析其對發展中國家經濟的深遠影響。
2       請探討全球化影響與教育機會的關聯，並分析文獻中的爭議點。
3        從歷史與當前數據的角度，分析技術進步對城市居民的影響。
4         從歷史與當前數據的角度，分析數位化對年輕一代的影響。
                   ...              
95        請分析醫療保健體系如何影響移民工人，並總結政策建議。
96      請根據網絡安全的最新研究，探討其對跨國公司的多層次影響。
97    描述數位化在不同地區的應用，並分析其對勞動力市場改善的影響。
98       從企業社會責任政策的角度，分析其對貧富差距的潛在影響。
99       描述自動化在不同地區的應用，並分析其對產業發展的影響。
Name: questions, Length: 100, dtype: object

In [27]:
for i in range(1):
    keywords, query = keyword_extraction(questions[i])
    print(f"{i}. questions:", questions[i])
    print("query:", query)
    print("keywords", keywords)
    print("\n")

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


0. questions: 從社會保障政策的角度，分析其對貧富差距的潛在影響。
query: 假設你是一位經濟學領域的學者，請根據提供的的資料，從社會保障政策的角度，分析其對貧富差距的潛在影響。**請用繁體中文回答**
keywords 社會保障政策 貧富差距




In [28]:
# 初始化 SentenceTransformer 模型
model = SentenceTransformer('intfloat/multilingual-e5-large')

predicted_keywords = []
for i in range(len(data)):
    llm_generated_keywords, _ = keyword_extraction(data["questions"][i])  # 假設有一個關鍵字提取函數
    predicted_keywords.append(llm_generated_keywords)
    if (i + 1) % 5 == 0:
        print(f"{i + 1} questions are done!")

# 這裡將 true_keywords 和 predicted_keywords 轉換為二進制格式
true_keywords = data["keywords"]

5 questions are done!


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


10 questions are done!
15 questions are done!
20 questions are done!
25 questions are done!
30 questions are done!
35 questions are done!
40 questions are done!
45 questions are done!
50 questions are done!
55 questions are done!
60 questions are done!
65 questions are done!
70 questions are done!
75 questions are done!
80 questions are done!
85 questions are done!
90 questions are done!
95 questions are done!
100 questions are done!


In [29]:
adjusted_predicted_keywords = []
for i in range(len(predicted_keywords)):
    adjusted_list = []
    # 將字串轉為列表
    true_kw_list = true_keywords[i].split()
    pred_kw_list = predicted_keywords[i].split()
    # 批量編碼
    true_embeddings = model.encode(true_kw_list, convert_to_tensor=True)
    pred_embeddings = model.encode(pred_kw_list, convert_to_tensor=True)
    for j, pred_kw in enumerate(pred_kw_list):
        # 計算每個 predicted_keyword 與 true_keywords 的相似度
        cos_similarities = util.cos_sim(pred_embeddings[j], true_embeddings).cpu().tolist()
        # 找到相似度最大值
        max_similarity = max(cos_similarities[0])
        if max_similarity > 0.9:
            max_idx = cos_similarities[0].index(max_similarity)
            adjust_kw = true_kw_list[max_idx]
            adjusted_list.append(adjust_kw)
        else:
            adjusted_list.append(pred_kw)
    # 保存結果
    adjusted_predicted_keywords.append(adjusted_list if adjusted_list else [])

In [30]:
# Convert string format to list format if needed
true_kw_lists = [kw.split() if isinstance(kw, str) else kw for kw in true_keywords]
pred_kw_lists = [kw.split() if isinstance(kw, str) else kw for kw in adjusted_predicted_keywords]

# Get unique keywords from both sets
all_keywords = set()
for kw_list in true_kw_lists + pred_kw_lists:
    all_keywords.update(kw_list)

# Create binary vectors
mlb = MultiLabelBinarizer(classes=list(all_keywords))
y_true = mlb.fit_transform(true_kw_lists)
y_pred = mlb.transform(pred_kw_lists)

# Calculate metrics
metrics = {
    'precision': precision_score(y_true, y_pred, average='micro'),
    'recall': recall_score(y_true, y_pred, average='micro'),
    'f1': f1_score(y_true, y_pred, average='micro'),
}


# Create a formatted results dataframe
results_df = pd.DataFrame({
    'Metric': list(metrics.keys()),
    'Score': list(metrics.values())
})

results_df

,Metric,Score
0,precision,0.770992
1,recall,0.939535
2,f1,0.846960


### test faithfulness

In [22]:
keywords = []
querys = []
retrieved_contents = []
responses =[]
for i in range(len(questions)):
    keyword, query = keyword_extraction(questions[i])
    print(f"{i + 1}.")
    print("query:", query)
    print("keywords", keyword)
    retrieved_content = retrieve(keyword)
    response = final_qa(keyword, retrieved_content, query)
    print("Final Answer:")
    print(response)   
    keywords.append(keyword)
    querys.append(query)
    retrieved_contents.append(retrieved_content)
    responses.append(response)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


1.
query: 假設你是一位經濟學領域的學者，請根據提供的的資料，從社會保障政策的角度，分析其對貧富差距的潛在影響。**請用繁體中文回答**
keywords 社會保障政策 貧富差距 社會保障政策貧富差距無第三個關鍵字無第三個關鍵字
Final Answer:



答案：

根據提供的資料，以下是我們對社會保障政策對貧富差距的潛在影響的分析：

1. 德國長期照顧保險政策：研究發現，德國長保固然在財政與服務供給等面向達到立法目的，但是長期而言卻在性別、階級與族群等面向皆造成擴大不平等差距之趨勢。

2. 臺灣國民年金政策：研究發現，國民年金政策將致使中低收入老人家庭的總消費支出因而下降9%。另外，所得收入者的性別亦被觀察到其將會對戶內資源的分配造成差異性的影響。

3. 臺灣社會福利支出的影響：研究發現，社會福利支出對於跨世代財政負擔的影響頗大，如果不考慮「消費支出層級」，則以需求因素影響力及貢獻度最大，其次為價格因素、參考因素、購買力因素以及偏好因素。

4. 臺灣社會住宅政策的影響：研究發現，社會住宅政策對其周邊房屋租金可能產生的影響頗大，尤其是在提供社會住宅的時間點與其周邊空間範圍大小下，社會住宅對周邊房屋租金的影響效果會隨著空間距離範圍擴大而減弱。

5. 大陸的扶貧政策：研究發現，大陸的扶貧政策在改革開放後逐步轉變為經濟發展為主，財政撥款、扶貧貸款、以工代賑及科教扶貧等政策都是主要措施，但在市場化、經濟效益的推展與追求層面上，政策與扶貧目標間的矛盾，以及幹群之間產生的問題，還有貧困線的介定不明所導致的政策瞄準及公平性等問題。

綜上所述，社會保障政策對貧富差距的潛在影響頗廣泛，需要深入分析和評估，以便於政策制定時避免誤區和錯失良機。


資訊不足。 


注意：根據提供的資料，我們只能看到一些研究報告和學術論文，但並不是所有的資料都能夠幫助我們了解社會保障政策對貧富差距的潛在影響。因此，我們需要更多的信息和資料來進行更加詳細的分析。 



如果您想知道哪些政策對貧富差距的影響最為密切，您可以查看以下幾篇文章：

*   林佳慧(Jia-Huey Lin)、郭祐誠(Yu-Chen Kuo)、周駿騰(Jun-Teng Zhou)(2024). 實現居住正義？社會住宅政策對周邊房屋租金之影響－以臺北市為例._臺灣經濟預測與政策_, _54_(2), 69

In [23]:
faithfulness_scores = []
for i in range(len(querys)):    
    score = calculate_faithfulness_score(retrieved_contents[i], querys[i], responses[i])
    faithfulness_scores.append(score)
    if (i + 1) % 5 == 0:
        print(f"{i + 1} questions are done!")

5 questions are done!
10 questions are done!
15 questions are done!
20 questions are done!
25 questions are done!
30 questions are done!
35 questions are done!
40 questions are done!
45 questions are done!
50 questions are done!
55 questions are done!
60 questions are done!
65 questions are done!
70 questions are done!
75 questions are done!
80 questions are done!
85 questions are done!
90 questions are done!
95 questions are done!
100 questions are done!


In [24]:
for i, score in enumerate(faithfulness_scores, start=1):
    print(f"問題 {i} 的 faithfulness_scores：{score}")

問題 1 的 faithfulness_scores：0.875
問題 2 的 faithfulness_scores：0.714
問題 3 的 faithfulness_scores：1.0
問題 4 的 faithfulness_scores：0.222
問題 5 的 faithfulness_scores：1.0
問題 6 的 faithfulness_scores：1.0
問題 7 的 faithfulness_scores：0.688
問題 8 的 faithfulness_scores：1.0
問題 9 的 faithfulness_scores：0.667
問題 10 的 faithfulness_scores：0.923
問題 11 的 faithfulness_scores：1.0
問題 12 的 faithfulness_scores：0.7
問題 13 的 faithfulness_scores：1.0
問題 14 的 faithfulness_scores：1.0
問題 15 的 faithfulness_scores：1.0
問題 16 的 faithfulness_scores：0.455
問題 17 的 faithfulness_scores：0.8
問題 18 的 faithfulness_scores：1.0
問題 19 的 faithfulness_scores：0.667
問題 20 的 faithfulness_scores：0.944
問題 21 的 faithfulness_scores：0.6
問題 22 的 faithfulness_scores：1.0
問題 23 的 faithfulness_scores：0.8
問題 24 的 faithfulness_scores：0.667
問題 25 的 faithfulness_scores：0.8
問題 26 的 faithfulness_scores：0.87
問題 27 的 faithfulness_scores：0.875
問題 28 的 faithfulness_scores：1.0
問題 29 的 faithfulness_scores：0.375
問題 30 的 faithfulness_scores：1.0
問題 31 的 faithfulness_sco

In [25]:
print(sum(faithfulness_scores)/len(faithfulness_scores))

0.7896299999999999


### test answer relevance

In [29]:
answer_relevance_scores = []
for i in range(len(responses)):
    pred_questions = question_generation(responses[i])
    score = calculate_answer_relevance_score(querys[i], pred_questions.content)
    answer_relevance_scores.append(score)
    if (i + 1) % 5 == 0:
        print(f"{i + 1} questions are done!")

5 questions are done!
10 questions are done!
15 questions are done!
20 questions are done!
25 questions are done!
30 questions are done!
35 questions are done!
40 questions are done!
45 questions are done!
50 questions are done!
55 questions are done!
60 questions are done!
65 questions are done!
70 questions are done!
75 questions are done!
80 questions are done!
85 questions are done!
90 questions are done!
95 questions are done!
100 questions are done!


In [35]:
for i, score in enumerate(answer_relevance_scores, start=1):
    print(f"Question {i} answer_relevance score: {round(score, 3)}")

Question 1 answer_relevance score: 0.943
Question 2 answer_relevance score: 0.927
Question 3 answer_relevance score: 0.921
Question 4 answer_relevance score: 0.898
Question 5 answer_relevance score: 0.919
Question 6 answer_relevance score: 0.939
Question 7 answer_relevance score: 0.908
Question 8 answer_relevance score: 0.938
Question 9 answer_relevance score: 0.93
Question 10 answer_relevance score: 0.938
Question 11 answer_relevance score: 0.924
Question 12 answer_relevance score: 0.948
Question 13 answer_relevance score: 0.943
Question 14 answer_relevance score: 0.911
Question 15 answer_relevance score: 0.914
Question 16 answer_relevance score: 0.941
Question 17 answer_relevance score: 0.937
Question 18 answer_relevance score: 0.931
Question 19 answer_relevance score: 0.922
Question 20 answer_relevance score: 0.941
Question 21 answer_relevance score: 0.937
Question 22 answer_relevance score: 0.935
Question 23 answer_relevance score: 0.921
Question 24 answer_relevance score: 0.912
Qu

In [36]:
average_relevance_scores = round(sum(answer_relevance_scores)/len(answer_relevance_scores), 3)
print(average_relevance_scores)

0.928
